In [1]:
#this defaults plots to matplotlib 1.0 formatting
import matplotlib.style
import matplotlib as mpl
#mpl.style.use('classic')
%matplotlib inline
mpl.rcParams['figure.dpi']= 150

# importing packages needed to run this script 
from subprocess import call
import pickle as pickle
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.cluster.hierarchy as sch

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import copy
import colorsys
import matplotlib.image as mpimg
from IPython.display import display
from IPython.display import Image
import scipy.cluster.hierarchy as sch
from scipy.stats import scoreatpercentile 
from pysca import scaTools as sca
# import mpld3
import pickle as pickle
from optparse import OptionParser
import sys
sys.path.append('../utilities/')

from graph_helpers import (
   gen_amino_acid_position_in_ref_sequence_graph,
   gen_clustered_similarity_matrix, 
   gen_unclustered_similarity_matrix,
   gen_pairwise_sequence_identities
)
from helpers import (
    show_processmsa_results,
    seq_count_check
)

%matplotlib inline

if not os.path.exists('../output/'):
    os.makedirs('../output/')  

In [2]:
input_pfam = '/home/bjohnson/rdb/data/Insig1pfamalign.txt'
output_pfam = '/home/bjohnson/rdb/data/Insig1pfamalign-annotated.txt'
pfam_seq = '/home/bjohnson/rdb/data/pfamseq.txt'
#construct a dictionary of phylogenetic groups
sca.AnnotPfam(input_pfam, output_pfam, pfam_seq=pfam_seq)
print('Executed')

Beginning annotation
Current step 0, key A0A672RDX5_SINGR
Current step 1, key A0A183A3Z3_9TREM
Current step 2, key A0A315V631_GAMAF
Current step 3, key A0A094HD51_9PEZI
Current step 4, key A0A2V1B028_9ASCO
Current step 5, key U7PQ59_SPOS1
Current step 6, key A0A1S3NY84_SALSA
Current step 7, key A0A507BYB9_9FUNG
Current step 8, key A0A3P6I7P0_9CEST
Current step 9, key T1EG13_HELRO
Current step 10, key A0A6P6ILR7_PUMCO
Current step 11, key A0A3M0KZN1_HIRRU
Current step 12, key A0A672RYJ6_SINGR
Current step 13, key A0A2I3M1C3_PAPAN
Current step 14, key J3NIF0_GAET3
Current step 15, key A0A2P8A3V9_9PEZI
Current step 16, key A0A151PDE1_ALLMI
Current step 17, key A0A091NX45_HALAL
Current step 18, key S8A8P3_DACHA
Current step 19, key A0A3N2Q7Y7_9PEZI
Current step 20, key G0WCX0_NAUDC
Current step 21, key A0A2Y9HAD7_NEOSC
Current step 22, key A0A384BW71_URSMA
Current step 23, key A0A197KDX7_9FUNG
Current step 24, key A0A6P6KGC5_CARAU
Current step 25, key Q0CLJ8_ASPTN
Current step 26, key A0A0

Current step 722, key A0A5N5X257_9EURO
Current step 723, key A0A197JBY2_9FUNG
Current step 724, key A0A420IFJ5_9PEZI
Current step 725, key J7S777_KAZNA
Current step 726, key A0A5J4NS64_9TREM
Current step 727, key A0A2I4CP23_9TELE
Current step 728, key A0A6P7LF01_BETSP
Current step 729, key H2QIL8_PANTR
Current step 730, key W3WSL5_PESFW
Current step 731, key A0A4Q4YSY0_9PEZI
Current step 732, key W6YCP1_COCCA
Current step 733, key A0A6P7HEQ8_9TELE
Current step 734, key A0A0L0H9X9_SPIPD
Current step 735, key K1S6Q0_CRAGI
Current step 736, key A0A0D2FZP7_9EURO
Current step 737, key INSI1_DANRE
Current step 738, key E3RCG2_PYRTT
Current step 739, key A0A7H8R8G5_9EURO
Current step 740, key A0A1S4AE05_TOBAC
Current step 741, key A0A369HB48_9HYPO
Current step 742, key A0A2K1QRX3_9PEZI
Current step 743, key A0A4X2L0M2_VOMUR
Current step 744, key A0A4Q4U0S1_9PEZI
Current step 745, key V3ZPY9_LOTGI
Current step 746, key A0A7N8XL29_9TELE
Current step 747, key A0A437C4Z6_ORYJA
Current step 748, k

In [9]:
path2aln_processed = '/home/bjohnson/rdb/data/Insig1pfamalign-annotated.txt'
path2out = '../output/'
command =  str('scaProcessMSA -a %s -d %s -i 0' % (path2aln_processed, path2out))
print(command)
call(command,shell = 'True')
print("Executed")

scaProcessMSA -a /home/bjohnson/rdb/data/Insig1pfamalign-annotated.txt -d ../output/ -i 0
Loaded alignment of 1392 sequences, 849 positions.
Checking alignment for non-standard amino acids
Aligment size after removing sequences with non-standard amino acids: 1387
Trimming alignment for highly gapped positions (80% or more).
1387
Alignment size post-trimming: 226 positions
Looking for PDBs in home/bjohnson/pySCA-master/data/
using provided reference index 0
A0A672RDX5_SINGR|Insulin-induced gene protein |Sinocyclocheilus grahami (Dianchi golden-line fish) (Barbus grahami)|Eukaryota,Metazoa,Chordata,Craniata,Vertebrata,Euteleostomi,Actinopterygii,Neopterygii,Teleostei,Ostariophysi,Cypriniformes,Cyprinidae,Cyprininae,Sinocyclocheilus.
here!
len refseq 182, len refpos 226, len algseq 226, len pairalg 226, len gloalg 226
Conducting sequence and position filtering: alignment size is 1387 seqs, 226 pos
ATS should also have 226 positions - ATS: 226
Keeping 975 sequences of 1387 sequences (after

In [11]:
dbname = "Insig1pfamalign-annotated.db"
pickle_path = '/home/bjohnson/rdb/output/Insig1pfamalign-annotated.db'
db = pickle.load(open(pickle_path,'rb'))
Dseq = db['sequence']

In [12]:
show_processmsa_results(Dseq)

*****Final processed alignment parameters****
Number of sequences: M = 956
Number of effective sequences: M' = 154
Number of alignment positions: L = 177
Number of positions in the ats: 177
Number of structure positions mapped: 177
Missing key distmat
****************************************


In [13]:
# check for sufficient sequences in alignment
seq_count_check(Dseq)

The number of effective sequences in this MSA is: 154
This alignment has a sufficient number of effective sequences for analysis with SCA.
Npos (L) = 177


In [14]:
# Apply scaCore to our processed alignment 
command = 'scaCore -i %s' % (pickle_path)
print (command)
call(command, shell = 'True')
db = pickle.load(open(pickle_path,'rb'))
Dsca = db['sca']
print("Executed")

scaCore -i /home/bjohnson/rdb/output/Insig1pfamalign-annotated.db
Computing the sequence projections.
Computing sequence similarity matrix.
Computing the SCA conservation and correlation values.
Computing matrix randomizations...
Randomizations complete, 10 trials, time: 0.4 minutes
Calculations complete, writing to database file /home/bjohnson/rdb/output/Insig1pfamalign-annotated
Executed


In [15]:
# run scaSectorID 
command = 'scaSectorID -i %s' % (pickle_path)
print (command)
call(command, shell = 'True')

# Load and store results of scaSectorID
db = pickle.load(open(pickle_path,'rb'))
Dsect = db['sector'] 
print("Executed")

scaSectorID -i /home/bjohnson/rdb/output/Insig1pfamalign-annotated.db
Selected kpos=6 significant eigenmodes.
Calculations complete, writing to database file /home/bjohnson/rdb/output/Insig1pfamalign-annotated
Executed
